# Step 1: Set up

In [65]:
import gc
import xgboost as xgb 
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import csr_matrix
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA, TruncatedSVD
import os  # for Macbook
os.environ['KMP_DUPLICATE_LIB_OK']='True'

### Load Data

In [66]:
print('Started Loading data...')
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
print('Done!')

Started Loading data...
Done!


# Step 2: clean data

### Rename the columns and extract the labels

In [67]:
print(train.columns.shape)
target = train['TARGET']
train.drop(['TARGET'], axis = 1, inplace = True)
train.columns.shape

(371,)


(370,)

### Remove Duplicated Features, including zero columns

In [68]:
print('Removing Duplicated Columns....')
remove = []
c = train.columns
for i in range(len(c)-1):
    v = train[c[i]].values
    for j in range(i+1, len(c)):
        if np.array_equal(v, train[c[j]].values):
            remove.append(c[j])
print(np.shape(remove))
train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)
print('Done!')

Removing Duplicated Columns....
(590,)
Done!


### Remove constant features

In [69]:
print('Removing constant columns....')
remove = []
for col in train.columns:
    if len(train[col].unique()) == 0:
        remove.append(col)
print(np.shape(remove))
train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)
print('Done!')

Removing constant columns....
(0,)
Done!


### Add a feature that counts number of assets

In [70]:
original_features = train.columns[1:] 

In [71]:
print(train.shape)
train['SumZeros'] = (train[original_features] == 0).sum(axis=1)
test['SumZeros'] = (test[original_features] == 0).sum(axis=1)
train.shape

(76020, 308)


(76020, 309)

In [72]:
train.columns

Index(['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3',
       ...
       'saldo_medio_var33_hace2', 'saldo_medio_var33_hace3',
       'saldo_medio_var33_ult1', 'saldo_medio_var33_ult3',
       'saldo_medio_var44_hace2', 'saldo_medio_var44_hace3',
       'saldo_medio_var44_ult1', 'saldo_medio_var44_ult3', 'var38',
       'SumZeros'],
      dtype='object', length=309)

# Step 3. Generate features 

### Option 1. Generate PCA features

In [17]:
#df = pd.concat([train, test], axis = 0)
#df.shape
#n_train = train.shape[0] # for split the df back into train, test later
#n_test = test.shape[0]

def generate_PCA_feature(train, test, original_feature, n_components = 4):
    """
    we fit a PCA decomposition model to training set of the shape
    : n_train_samples * n_features
    and transform on both training set and testing test. 
    """
    pca = PCA(n_components = n_components)
    # we need to normalize the data before fitting
    train_projected = pca.fit_transform(normalize(train[original_features], axis=0))
    test_projected = pca.transform(normalize(test[original_features], axis=0))
    for i in range(1, n_components + 1):
        name = 'PCA{:02d}'.format(i)
        train[name] = train_projected[:, i - 1]
        test[name] = test_projected[:, i - 1]
    return train, test
print('Generating PCA features')
train, test = generate_PCA_feature(train, test, original_features, n_components = 5)
print('Done!') 

Generating PCA features
Done!


In [18]:
train.columns #  more features added, as expected

Index(['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3',
       ...
       'saldo_medio_var44_hace2', 'saldo_medio_var44_hace3',
       'saldo_medio_var44_ult1', 'saldo_medio_var44_ult3', 'var38', 'PCA01',
       'PCA02', 'PCA03', 'PCA04', 'PCA05'],
      dtype='object', length=375)

### Option 2. Truncated SVD features for nonlinear clustering

In [73]:
# we perform cross validation to set the n_components
def generate_SVD_feature(train, test, original_features, n_components = 5):
    """
    we fit a PCA decomposition model to training set of the shape
    : n_train_samples * n_features
    and transform on both training set and testing test. 
    """
    svd = TruncatedSVD(n_components = n_components)
    # we need to normalize the data before fitting
    train_projected = svd.fit_transform(train[original_features])
    test_projected = svd.transform(test[original_features])
    for i in range(1, n_components + 1):
        name = 'SVD{:02d}'.format(i)
        train[name] = train_projected[:, i - 1]
        test[name] = test_projected[:, i - 1]
    return train, test
print('Generating SVD features')
train, test = generate_SVD_feature(train, test, original_features, n_components = 8)
print('Done!')

Generating SVD features
Done!


In [74]:
train.columns # 5 more features added, as expected

Index(['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3',
       ...
       'var38', 'SumZeros', 'SVD01', 'SVD02', 'SVD03', 'SVD04', 'SVD05',
       'SVD06', 'SVD07', 'SVD08'],
      dtype='object', length=317)

# Step 4. Train model by XGBoost

### 10-fold cross validation splits

In [75]:
split = 10
skf = StratifiedKFold(n_splits = split)
skf.get_n_splits(train, target)

10

### Set parameters

In [87]:
num_boost_round = 500
params = {}
params["objective"] = "binary:logistic"
params["eta"] = 0.03 # 如同学习率
params["subsample"] = 1  # 随机采样训练样本 训练实例的子采样比
params["colsample_bytree"] = 0.7 # 生成树时进行的列采样
params["silent"] = 0 #设置成1则没有运行信息输出，最好是设置为0.是否在运行升级时打印消息。
params["max_depth"] = 5 # 构建树的深度，越大越容易过拟合
params["min_child_weight"] = 1
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
params["eval_metric"] = "auc"

In [88]:
train['target'] = target # add back the label
features = train.columns[1:-1] # exclude ID

In [84]:
features

Index(['var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3', 'imp_op_var40_ult1',
       ...
       'var38', 'SumZeros', 'SVD01', 'SVD02', 'SVD03', 'SVD04', 'SVD05',
       'SVD06', 'SVD07', 'SVD08'],
      dtype='object', length=316)

In [85]:
train.columns

Index(['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3',
       ...
       'SumZeros', 'SVD01', 'SVD02', 'SVD03', 'SVD04', 'SVD05', 'SVD06',
       'SVD07', 'SVD08', 'target'],
      dtype='object', length=318)

In [91]:
train_preds = None
test_preds = None
xgb_classifiers = [] # List[[clf, score]]
index = 0
for train_index, test_index in skf.split(train, target):
    print('Fold:', index + 1)
    index = index + 1
    X_train, X_test = train.iloc[train_index], train.iloc[test_index] # split training set into'train', 'cross-validation' sets
#    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

    # for xgb classifier, we transform them into DMatrix format
    D_train = xgb.DMatrix(
                    csr_matrix(X_train[features]),
                    X_train.target.values,
                    silent = True)
#    print(D_train.num_col())
#    print(D_train.num_row())
    
    D_test = xgb.DMatrix(
                    csr_matrix(X_test[features]),
                    X_test.target.values,
                    silent = True)
    watchlist = [(D_test, 'test'), (D_train, 'train')]
#    print(D_test.num_col())
#    print(D_test.num_row())
  
    # fit the classfier now
    clf = xgb.train(params, D_train, num_boost_round,
                    evals = watchlist, early_stopping_rounds = 250)
    
    test_prediction = clf.predict(D_test)
    print('Blind Log Loss:', log_loss(X_test.target.values, test_prediction))
    score = roc_auc_score(X_test.target.values, test_prediction)
    print('Blind ROC:', score)
    
    
    del X_train, X_test, D_train, D_test
    gc.collect()
    print('finished a training model')
    print('fitting on full data set now...')
    
    D_full_train = \
        xgb.DMatrix(csr_matrix(train[features]),
                    train.values,
                    silent = True)
    D_full_test = \
        xgb.DMatrix(csr_matrix(test[features]),
                    silent = True)
    if(train_preds is None):
        train_preds = clf.predict(D_full_train)
        test_preds = clf.predict(D_full_test)
    else:
        train_preds *= clf.predict(D_full_train) # we manually perform an average of the results
        test_preds *= clf.predict(D_full_test)
    xgb_classifiers.append([clf, 'with auc score: {:10f}'.format(score)])
    del D_full_train, D_full_test, clf
    gc.collect()


print('Done!')

Fold: 1
[0]	test-auc:0.794505	train-auc:0.811918
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 250 rounds.
[1]	test-auc:0.804275	train-auc:0.819368
[2]	test-auc:0.804682	train-auc:0.82001
[3]	test-auc:0.805642	train-auc:0.821869
[4]	test-auc:0.806874	train-auc:0.822753
[5]	test-auc:0.811785	train-auc:0.827749
[6]	test-auc:0.810877	train-auc:0.82839
[7]	test-auc:0.821164	train-auc:0.831997
[8]	test-auc:0.816336	train-auc:0.832987
[9]	test-auc:0.815987	train-auc:0.832718
[10]	test-auc:0.819946	train-auc:0.836599
[11]	test-auc:0.816593	train-auc:0.836028
[12]	test-auc:0.817316	train-auc:0.836937
[13]	test-auc:0.814823	train-auc:0.837431
[14]	test-auc:0.815583	train-auc:0.837754
[15]	test-auc:0.815982	train-auc:0.83873
[16]	test-auc:0.817481	train-auc:0.840044
[17]	test-auc:0.817617	train-auc:0.841075
[18]	test-auc:0.819919	train-auc:0.841812
[19]	test-auc:0.819736	train-auc:0.841739
[20]	test-auc:0.81785

[191]	test-auc:0.835791	train-auc:0.875656
[192]	test-auc:0.83584	train-auc:0.875815
[193]	test-auc:0.835866	train-auc:0.875952
[194]	test-auc:0.835864	train-auc:0.876096
[195]	test-auc:0.836068	train-auc:0.876158
[196]	test-auc:0.83604	train-auc:0.876238
[197]	test-auc:0.836104	train-auc:0.87636
[198]	test-auc:0.836064	train-auc:0.876559
[199]	test-auc:0.836096	train-auc:0.876684
[200]	test-auc:0.836178	train-auc:0.876781
[201]	test-auc:0.836235	train-auc:0.876922
[202]	test-auc:0.836232	train-auc:0.877028
[203]	test-auc:0.836173	train-auc:0.877204
[204]	test-auc:0.836184	train-auc:0.877414
[205]	test-auc:0.836391	train-auc:0.87752
[206]	test-auc:0.836276	train-auc:0.87775
[207]	test-auc:0.836345	train-auc:0.877892
[208]	test-auc:0.836468	train-auc:0.878055
[209]	test-auc:0.836547	train-auc:0.878186
[210]	test-auc:0.836465	train-auc:0.87849
[211]	test-auc:0.836488	train-auc:0.878607
[212]	test-auc:0.836507	train-auc:0.878637
[213]	test-auc:0.83643	train-auc:0.878739
[214]	test-auc:0.8

[383]	test-auc:0.837232	train-auc:0.889412
[384]	test-auc:0.837222	train-auc:0.889431
[385]	test-auc:0.837171	train-auc:0.889455
[386]	test-auc:0.837136	train-auc:0.88951
[387]	test-auc:0.837193	train-auc:0.889567
[388]	test-auc:0.837155	train-auc:0.889613
[389]	test-auc:0.83717	train-auc:0.889637
[390]	test-auc:0.837222	train-auc:0.889678
[391]	test-auc:0.83721	train-auc:0.889713
[392]	test-auc:0.837238	train-auc:0.889742
[393]	test-auc:0.837258	train-auc:0.889773
[394]	test-auc:0.837258	train-auc:0.889797
[395]	test-auc:0.837243	train-auc:0.889855
[396]	test-auc:0.837203	train-auc:0.889875
[397]	test-auc:0.837189	train-auc:0.889892
[398]	test-auc:0.837207	train-auc:0.889917
[399]	test-auc:0.837182	train-auc:0.889945
[400]	test-auc:0.837141	train-auc:0.889962
[401]	test-auc:0.837192	train-auc:0.890013
[402]	test-auc:0.837191	train-auc:0.890053
[403]	test-auc:0.837187	train-auc:0.890071
[404]	test-auc:0.837186	train-auc:0.890136
[405]	test-auc:0.837173	train-auc:0.890153
[406]	test-auc

/Users/xiaoguai/anaconda3/envs/sklearn/lib/python3.6/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


Fold: 2
[0]	test-auc:0.803169	train-auc:0.819798
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 250 rounds.
[1]	test-auc:0.805712	train-auc:0.82342
[2]	test-auc:0.805583	train-auc:0.824615
[3]	test-auc:0.809702	train-auc:0.829309
[4]	test-auc:0.812242	train-auc:0.8302
[5]	test-auc:0.815621	train-auc:0.831675
[6]	test-auc:0.816899	train-auc:0.834127
[7]	test-auc:0.816016	train-auc:0.834295
[8]	test-auc:0.817043	train-auc:0.835006
[9]	test-auc:0.816831	train-auc:0.835023
[10]	test-auc:0.8172	train-auc:0.836127
[11]	test-auc:0.817449	train-auc:0.836487
[12]	test-auc:0.818549	train-auc:0.837871
[13]	test-auc:0.816793	train-auc:0.837459
[14]	test-auc:0.817463	train-auc:0.838143
[15]	test-auc:0.818964	train-auc:0.838793
[16]	test-auc:0.819139	train-auc:0.839654
[17]	test-auc:0.819473	train-auc:0.840267
[18]	test-auc:0.819894	train-auc:0.840505
[19]	test-auc:0.82068	train-auc:0.840746
[20]	test-auc:0.819926	t

...

Fold: 10
[0]	test-auc:0.789616	train-auc:0.816004
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 250 rounds.
[1]	test-auc:0.790562	train-auc:0.819802
[2]	test-auc:0.795208	train-auc:0.82033
[3]	test-auc:0.80715	train-auc:0.829056
[4]	test-auc:0.808353	train-auc:0.831517
[5]	test-auc:0.81153	train-auc:0.836311
[6]	test-auc:0.811617	train-auc:0.8374
[7]	test-auc:0.810582	train-auc:0.83743
[8]	test-auc:0.811425	train-auc:0.839191
[9]	test-auc:0.812217	train-auc:0.839717
[10]	test-auc:0.813057	train-auc:0.840308
[11]	test-auc:0.81521	train-auc:0.840284
[12]	test-auc:0.814539	train-auc:0.840384
[13]	test-auc:0.814164	train-auc:0.840228
[14]	test-auc:0.814298	train-auc:0.84058
[15]	test-auc:0.816481	train-auc:0.841578
[16]	test-auc:0.815817	train-auc:0.841707
[17]	test-auc:0.816803	train-auc:0.842468
[18]	test-auc:0.817042	train-auc:0.842915
[19]	test-auc:0.816523	train-auc:0.842919
[20]	test-auc:0.816166	tr

[191]	test-auc:0.8257	train-auc:0.876626
[192]	test-auc:0.825688	train-auc:0.876771
[193]	test-auc:0.825776	train-auc:0.876913
[194]	test-auc:0.825828	train-auc:0.877028
[195]	test-auc:0.825944	train-auc:0.877198
[196]	test-auc:0.825887	train-auc:0.877354
[197]	test-auc:0.825855	train-auc:0.877458
[198]	test-auc:0.825958	train-auc:0.877583
[199]	test-auc:0.825968	train-auc:0.877745
[200]	test-auc:0.826056	train-auc:0.877849
[201]	test-auc:0.82597	train-auc:0.878006
[202]	test-auc:0.826004	train-auc:0.878082
[203]	test-auc:0.826077	train-auc:0.878257
[204]	test-auc:0.826001	train-auc:0.878554
[205]	test-auc:0.826106	train-auc:0.878633
[206]	test-auc:0.826111	train-auc:0.878869
[207]	test-auc:0.826149	train-auc:0.878989
[208]	test-auc:0.826016	train-auc:0.879118
[209]	test-auc:0.826013	train-auc:0.879188
[210]	test-auc:0.826032	train-auc:0.879302
[211]	test-auc:0.826007	train-auc:0.879329
[212]	test-auc:0.826006	train-auc:0.879371
[213]	test-auc:0.826062	train-auc:0.879537
[214]	test-auc

[383]	test-auc:0.82788	train-auc:0.890334
[384]	test-auc:0.827819	train-auc:0.890429
[385]	test-auc:0.827829	train-auc:0.890474
[386]	test-auc:0.827864	train-auc:0.890517
[387]	test-auc:0.827862	train-auc:0.890562
[388]	test-auc:0.82788	train-auc:0.890571
[389]	test-auc:0.827888	train-auc:0.890589
[390]	test-auc:0.827887	train-auc:0.890615
[391]	test-auc:0.827882	train-auc:0.890631
[392]	test-auc:0.827961	train-auc:0.890672
[393]	test-auc:0.82795	train-auc:0.890686
[394]	test-auc:0.827956	train-auc:0.890728
[395]	test-auc:0.827929	train-auc:0.890822
[396]	test-auc:0.827868	train-auc:0.890857
[397]	test-auc:0.827942	train-auc:0.890918
[398]	test-auc:0.827919	train-auc:0.891005
[399]	test-auc:0.827951	train-auc:0.891027
[400]	test-auc:0.827941	train-auc:0.891048
[401]	test-auc:0.827933	train-auc:0.891067
[402]	test-auc:0.827904	train-auc:0.891092
[403]	test-auc:0.827908	train-auc:0.891136
[404]	test-auc:0.827936	train-auc:0.891147
[405]	test-auc:0.828024	train-auc:0.891198
[406]	test-auc

/Users/xiaoguai/anaconda3/envs/sklearn/lib/python3.6/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


Done!


### save the xgb_classifiers lists for later ensemble

In [19]:
pickle.dump(xgb_classifiers, open('output/xgboost_classifier_param1.dat', 'wb'))

In [20]:
#data = pickle.load(open('output/xgboost_classifier_param1.dat', 'rb'))

### output

In [92]:
train_preds = np.power(train_preds, 1./split)
test_preds = np.power(test_preds, 1./split)
print('Average Log Loss:', log_loss(train.target.values, train_preds))
print('Average ROC:', roc_auc_score(train.target.values, train_preds))

Average Log Loss: 0.12014701864960121
Average ROC: 0.8911137007243636


In [22]:
submission = pd.DataFrame({"ID": train.ID,
                           "TARGET": train.target,
                           "PREDICTION": train_preds})

submission.to_csv("output/simplexgbtrain.csv", index = False)
submission = pd.DataFrame({"ID": test.ID, "TARGET": test_preds})
submission.to_csv("output/simplexgbtest.csv", index = False)
print('Finish')

Finish
